# Variant calling of E.coli RNAseq data: A workflow

## Introduction

In this notebook we will take RNAseq data from a paper  named "Mammalian histones facilitate antimicrobial synergy by disrupting the bacterial proton gradient and chromosome organization". The paper can be accessed at:

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7403156/

In this paper it is studied the possiblity that histone H2A in *E. coli* and *S. aureus* acts as an antimicrobial agent. In the paper it is stated that H2A can depolarize bacterial membrane and impair its recovery, and with the combination of LL-37, a pore inducer in bacterial membrane, it promotes the antimicrobial effect of H2A.
One of the things the do study the possible effect of H2A in combination of LL-37 of the organism, is adding different quantities of H2A in cultures of the strain K-12 substrain MG1655 of *E. coli*  and doing an RNAseq after exposition to H2A. For more detailed information you can read the paper linked above. This paper has the sequences resulting of the RNAseq sequencing uploaded at GEOdataset, with more information about the experiment and sequencing:

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE142755

At the GEO publication of this paper we can see that they did 4 experimental groups that were replicated 3 times. One group was the one untreated, serving as a reference and the next three were treated with 10 μg/mL, 50 μg/mL and 100 μg/mL of H2A respectively. We also have to take in consideration that the sequencing was performed in the platform Illumina HiSeq 2500. If the raw RNAseq data is not uploaded at this GEO accesion, we can try reaching it at the ENA browser, introducing the BioProject ID of the GEO Dataset, being in this case PRJNA598209. The link to the RNAseq data is:

https://www.ebi.ac.uk/ena/browser/view/PRJNA598209

Here we can see more info about the RNAseq data obtained in the study, and we have the possibility of downloading the gzipped fastq sequences of each sample and its replicates. If we download all the samples the downloaded files will be in a zip file to compress all the files.

## Tools needed

To succesfuly do a variant calling of RNAseq data we need to have a variety of tools that may vary depending on the type of data, precission needed in the analysis, specs of your computer and available storage space, etc. In my case I am using a "mid-quality" specs referring to RNAseq analysis, as well with low storage space, so I will need tools that require low amount of specs and storage space when necessary.   

The specific tools used in this variant calling workflow are:
* bash terminal
* hisat2
* fastqc
* samtools
* bcftools
* IGV


For most of the tools we will need to use the linux terminal to install its dependencies and execute them properly, so bash knowledge is required to succesfuly do this variant calling.


## Preparing files and folders

First we can start creating a folder of the analysis, with subfolders that will contain the starting files, reports, intermediate files and the final VCF files. In my case I create the **proj2** file, and inside it I will create the subfolders **aligned** **fastq_zip** **ref** **reports** and **results**. Also, inside the folder **aligned** I will create the subfolders **sam** and **bam** and in the folder **results** the subfolders **bcf** and **vcf**. As we will be doing the workflow in this notebook we will see what we will add to each folder, but its names are usually self-conclusive. To create these directories we can use the command **mkdir** with the names of the files, or create them using the OS interface, but it's recommended to practice whith bash commands.


To move the zip file usually named *ena_vile.zip* to the desired folder we can use the command **mv** or use the OS interface. We will move the zip file to the folder **fastq_zip**, this way we will have stored all the fastq_zip files in this folder.

mv home/download/ena_file.zip home/Programs/proj2/fastq_zip

Note that it's useful for tracking to change the zip name to one informative, like h2a.zip. We can do this with the same **mv** command, changing only the name without touching the directory

mv home/Programs/proj2/fastq_zip/ena_file.zip home/Programs/proj2/fastq_zip/h2a.zip

As stated in the study, the reference genome used was from *E. coli*, specifically its string K-12 and substring MG1655. To download this reference genome we go to the NCBI webpage, and in the nucleotide section we type the specie, string and substring, finding the complete genome. Then we can "Send to" a File  as FASTA, that we will move to the **ref** folder when finished downloading. The specific link to find this genome:

https://www.ncbi.nlm.nih.gov/nuccore/NC_000913

To unzip a zip file we can use the command **unzip** and indicate the file to unzip. If we are in the same folder we don't need to add the path to the file name:

unzip h2a.zip

Now we have in the **fastq_zip** folder all the fastq samples in gz format, another type of format to compress data. This format is not needed to be uncompressed due to the tools that we will be using are able to read the fastq sequences in this format. The problem we find here is that each sample is in its own subfolder so it would be harder to do analyses entering each subfolder separately to do the variant calling workflow. To have all the fastq files together outside its subfolders we can move them with the **mv** command specifying that we want all files of each subfolder in the folder **fastq_zip**:

mv fastq_zip/\*/\* fastq_zip

After having all the fastq.gz files in the folder we can remove all the subfolders that will be now empty, and if we want we can remove also the original **h2a.zip** file, as it will be no longer useful. We can remove this files with the command **rm**.

## Index the reference genome

We need to index the reference genome to be able to use it later for mapping the samples of the study. To index the *E. coli* genome that we previously downloaded we need to use **hisat2**, an indexer and aligner tool that is fast and effective enough.

With the function **hisat2-build** we “build” the index of the reference genome named ecoli.fasta and name it with the prefix ecoli, so the indexes will be “ecoli.x.ht2” where x is an index number (indexing things..):

hisat2-build ecoli.fasta ecoli ecoli

## Quality of fastq files

To assess the quality of the fastq files of the different samples of the study, we can use the tool **fastqc**. This tool analyses the files and does a QC analysis, giving as output a zip file with all the analysis done in txt  format, images and html files, and an html file that lets us check with graphics the quality of the fastq file. To use fastqc we need to use simply the command **fastqc** and indicating the file or files that we want to analyse. We want to analyse all the fastq files that we have, so knowing that they all end with ".gz" we can ask to do all analyses in one command:

fastqc \*.gz

To have all the report files organized in one folder we can move them to the previously created **reports** folder, knowing that the outputs of the reports have the extensions .zip and .html: 

mv home/Programs/proj2/fastq_zip/\*.html home/Programs/proj2/reports/

mv home/Programs/proj2/fastq_zip/\*.zip home/Programs/proj2/reports/

We can study a lot of the parameters that appear at the different reports, but at this moment we only want to know if the fastq files are reliable enough to do a variant calling analysis with trusty results. Looking at the different reports we find that the "per base sequence content", "Sequence Length Distribution" and "Overrepresented sequences" are marked as warning in all the fastq files. Checking on the different warnings we don't see any problem that may affect the posterior analysis, so we don't need to worry at the moment. Also, we have an error in all the fastq files regarding the "Sequence Duplication Levels". Investigating about that we learned that the sequence duplication levels in RNAseq can vary due to its sequencing tending to duplicate some sequences, so it shouldn't be a problem during the analysis.

## Alignment

### Obtaining the sam file

We use directly the hisat2 aligner without trimming as I have seen that the difference with the use of trimmomatic or not doesn’t affect significantly at the final result. Here is the paper where I base this hypothesis:

https://academic.oup.com/nargab/article/2/3/lqaa068/5901066

The hisat2 aligner needs different parameters to aling correctly the sequences. First of all it needs the indexed reference genome, so we need to add its path after the parameter -x. Another parameter needed is the path to the fastq file, named as -U because we use an unpaired sequence. The last parameter needed is where we want to output the file that contains the alignment and in which format. We want the files in SAM format, so we use the -S parameter and add the path we want to save the .sam file.

hisat2 -x ref_hisat/ecoli -U fastq_zip/SRR10805236.fastq.gz -S aligned/sam/aln_SRR10805236.sam

### Passing from sam to bam

To do the variant calling we need pass the sam file to binary (bam), and sort the bam file after that. To do so we can use the tool **samtools**, designed to do these tasks (it can do a great variety of other tasks with sam files, but for this analysis we need it only for this). With the command **samtools view** we can convert a sam file to bam, needing only to pass the parameters -S (input SAM) and -b (output BAM), assigning a name and the path of the folder "sam" and "bam" respectively.

samtools view -S -b aligned/sam/aln_SRR10805236.sam > aligned/bam/aln_SRR10805230.bam

To sort the bam file we use the command **samtools sort** specifying the output with the -o parameter:

samtools sort -o aligned/bam/s_aln_SRR10805230.bam aligned/bam/aln_SRR10805230.bam

## BCF and VCF files

To change the file BAM file to BCF we need to use a new tool named **bcftools**. This tool let us do variant calling using the bam files and the reference genome. First of all with the command **bcftools mpileup** we can produce a BCF file contining genotype likelihoods for the specific BAM alignment. To do so we use different parameters to specify how we want the output. The "b" letter followed by the -O parameter indicates that we want an output in bcf format, and with the -o parameter we can specify which BAM file we will be using. Lastly with the -f parameter we indicate the reference sequence to be used to calculate the genotype likelihoods. Also we need to specify the name of the output file and its path, that will be in the folder **results** and subfolder **bcf**.

bcftools mpileup -O b -o results/bcf/SRR10805230.bcf -f ref/ecoli.fasta aligned/bam/s_aln_SRR10805230.bam

With the bcf file obtained we can call the variants obtaining as a result a vcf file, using again bcftools. Specifically, we will be using the command **bcftools call**, In this command we will add the parameters v, m and o to output only variant sites, call also multiallelic variants and specify the output path and name respectively.

bcftools call -vm -o results/vcf/SRR10805230.vcf results/bcf/SRR10805230.bcf

Lastly, we filter the vcf file with the command **bcftools filter** specifying the input and output.

bcftools filter results/vcf/SRR10805230.vcf > results/vcf/final_SRR10805230.vcf

With this file we can already see the variants called from the sample named SRR10805230 in a readble format, but there are a lot of SNPs detected of different depth in all the genome, so it would be advisable to look for specific regions to study.

## Viewing SNPs with IGV

To be able to see the SNPs of the sequence in a more visual format we can use the IGV tool, but for that we need to index specifically the reference genome and the bam file used. To do so we use the command **samtools index** with the sorted and aligned BAM file obtained previously. This will generate a bai file that will be used automatically when trying to view this specific bam file.

To view correctly the variants in IGV we have to load first the reference genome, being in this case "ecoli.fasta". Then we load the specific bam file and vcf file. Due to the size of the genome it will appear at first that there is no data to be displayed, but zooming in several times it will show the different regions of the genome, the bam file showing the regions sequenced, the SNPs found and other information like depth coverage or quality of the sequence.

## Automatising workflow

Having seen all the process and checking that it works properly in each step of the workflow, we can now automatise the Variant Calling of all the samples downloaded from the study. We can do that by designing a simple bash script that will execute all the necessary commands to go from the fastq.gz file to the vcf file of the sample, looping through each sample found at the folder **fastq_zip** folder. The full bash script is names as **variant_calling.sh**, bu we will explain in parts the process of the script to do the workflow.

First of all we specify to detect this as a bash script. Then we use **set -e** to stop the execution of a script if a command during the workflow has an error.

We add paths of the working directory of the genome and the genome file to facilitate indexing the genome with hisat and for later uses. Note that we don't have to loop through files because this has to be done only one time in all the workflow.

We start a for loop where we indicate the path of the variable *fastq* to be a fastq.gz file of all that are at the folder **fastq_zip**. Doing this we have a dynamic variable, that will change everytime the loop is ended. We echo the current fastq file name to report during the script in which file is working at the moment. Also, we take the base name of the file to facilitate the correct naming of all the files using the unique identifier that is its own base name. To check the identifier we make an echo of it.

Now we create variables that will be used later to save the different outputs obtained during the pipeline. We use the "base" variable to have differentiated names for each sample. Note that we change some file names to give more information than previously, but this doesn't impact any aspect of the pipeline. 

This last piece of code has all the commands explained previously put together, with the difference that we echo through the terminal every stage of the pipeline and we use the different variables created previously to easily use the input file and save correctly the output file, maintaining the files organized and correctly named. When the vcf files of all the samples have been obtained the bash script ends automatically.

It's important to note that the pipeline will start the loop in alphabetical order, starting in this case with **SRR10805225.fastq.gz** and ending with **SRR10805236.fastq.gz**. Also note that the loop goes from start to end in each file, meaning that it doesn't work in parallel with all the files in each step. 

The process of obtaining fastqc files of the samples has not been added in this script, but this has been done in purpose. The step of checking the quality of the samples has to be done before starting this pipeline, and depending on the quality of the fastq files it may be necessary to tweak the commands and add parameters or even it can advisable not to do the variant calling pipeline. Also it's better to manually index and prepare the data to view in IGV due to the possibility of not being necessary in the study and generating files that won't be used.

I have knowledge of the simplicity of the workflow and bash script, but I am also practising to do these pipelines, having to research about it by myself and trying to replicate some online pipelines with new data to get more experience. That being said, if you find an incorrect interpretation of the data, commands or anything else, do not hesitate to tell me. I appreciate all the feedback, as it will help me to grow and hone my skills in this field.

## References

Here I leave the links of information about RNAseq and pipelines that I used and tried to replicate to create this basic pipeline.

* https://sbc.shef.ac.uk/rna-seq-in-galaxy/01-pre-processing.nb.html
* https://www.ebi.ac.uk/sites/ebi.ac.uk/files/content.ebi.ac.uk/materials/2014/140217_AgriOmics/dan_bolser_snp_calling.pdf
* https://datacarpentry.org/wrangling-genomics/
* http://pedagogix-tagc.univ-mrs.fr/courses/ngs_galaxy/pdf_files/RNA-Seq_tutorial.pdf